In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_predict, cross_val_score, KFold, StratifiedKFold, train_test_split
from eigenface_project import eigenface_project
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
# Directory paths
train_dir = os.path.expanduser('../train_balanced')

# Define a function to load the balanced data
def load_images_from_directory(directory):
    images = []
    labels = []
    for label in os.listdir(directory):
        label_path = os.path.join(directory, label)
        if os.path.isdir(label_path):
            for file in os.listdir(label_path):
                file_path = os.path.join(label_path, file)
                image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
                images.append(image)
                labels.append(label)
    return images, labels

# Load training and testing images
train_images, train_labels = load_images_from_directory(train_dir)

print(f"Loaded {len(train_images)} training images.")

In [ ]:
print({x: train_labels.count(x) for x in set(train_labels)})
print(np.array(train_images).shape)

In [ ]:
# set up X, y
Class = np.array(train_labels)
train_images = np.array(train_images)

In [ ]:
# project the image data on the eigen vectors
train_images_project=eigenface_project(train_images)

In [ ]:
# define a random forest function, and search the best parameters

def auto_random_forest(
    X,
    y, 
    test_size=0.2,
    random_state=2024,
    n_estimators=100,
    cv=10
):
    """
    Implement Random Forest with different max_features settings and find optimal value
    
    Parameters:
    X, y: pandas DataFrame containing features and target
    test_size: proportion of data to use for testing
    random_state: random state for reproducibility
    n_estimators: number of trees in forest
    cv: number of cross-validation folds
    """
    
    # number of features
    n_features = np.array(X).shape[1]
    
    # Create different max_features values to try
    # Include common options and some specific numbers
    max_features_options = [
        "log2",
        "sqrt",
        0.1,     # 10% of features
    ]
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=test_size, 
        random_state=random_state,
        stratify=y
    )
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Create parameter grid
    param_grid = {
        'max_features': max_features_options,
        'min_samples_split': [2, 5, 10, 20, 30, 40, 50],
        'min_samples_leaf': [2, 5, 10, 20, 30, 40, 50]
    }
    
    # Initialize Random Forest
    rf = RandomForestClassifier(
        n_estimators=n_estimators,
        random_state=random_state
    )
    
    # Perform grid search
    grid_search = GridSearchCV(
        estimator=rf,
        param_grid=param_grid,
        cv=cv,
        scoring='accuracy',
        n_jobs=-1, # use all the CPU cores
        verbose=2  # detailed output monitoring progress updates
    )
    
    # Fit grid search
    grid_search.fit(X_train_scaled, y_train)
    
    # Get best model
    best_rf = grid_search.best_estimator_
    best_par = grid_search.best_params_
    
    # Make predictions with best model
    y_pred = best_rf.predict(X_test_scaled)
    
    # Evaluate different max_features settings
    max_features_results = []
    for max_feat in max_features_options:
        rf_temp = RandomForestClassifier(
            n_estimators=2000,
            max_features=max_feat,
            min_samples_split=best_par['min_samples_split'],
            min_samples_leaf=best_par['min_samples_leaf'],
            random_state=random_state
        )
        rf_temp.fit(X_train_scaled, y_train)
        train_score = rf_temp.score(X_train_scaled, y_train)
        test_score = rf_temp.score(X_test_scaled, y_test)
        
        # Convert max_feat to string for consistent display
        max_feat_str = str(max_feat)
        if isinstance(max_feat, float):
            max_feat_str = f"{max_feat:.1%} of features"
        
        max_features_results.append({
            'max_features': max_feat_str,
            'train_score': train_score,
            'test_score': test_score
        })
    
    # Compile results
    results = {
        'best_params': best_par,
        'best_score': grid_search.best_score_,
        'test_accuracy': accuracy_score(y_test, y_pred),
        'classification_report': classification_report(y_test, y_pred),
        'max_features_comparison': pd.DataFrame(max_features_results),
        'best_model': best_rf
    }
    
    return results

In [ ]:
random_forest_result = auto_random_forest(X=train_images_project, y=Class)

In [ ]:
# loop through and print the results
for key, value in random_forest_result.items():
    print(f"{key}: \n {value}")